* We have airport data in data directory in text file format.
* We are given airpirt id, name of airport, city name, IATA/FAA code, contry of airpirt, ICAO code, latitude, altitude, longitude, timezone DST, timezone in olson

### Import necessary package

In [2]:
import findspark
findspark.init('/home/purvil/spark-2.4.3-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('airport').getOrCreate()

### Read data in RDD

In [11]:
airport = spark.sparkContext.textFile("data/airports.text")

In [12]:
airport.take(2)

['1,"Goroka","Goroka","Papua New Guinea","GKA","AYGA",-6.081689,145.391881,5282,10,"U","Pacific/Port_Moresby"',
 '2,"Madang","Madang","Papua New Guinea","MAG","AYMD",-5.207083,145.7887,20,10,"U","Pacific/Port_Moresby"']

#### Find all airport located in the United States

In [13]:
import re
COMMA_DELIMITER = re.compile(''',(?=(?:[^"]*"[^"]*")*[^"]*$)''')

In [14]:
airportInUSA = airport.filter(lambda line:COMMA_DELIMITER.split(line)[3] == "\"United States\"")

In [15]:
airportInUSA.take(2)

['6891,"Putnam County Airport","Greencastle","United States","4I7",\\N,39.6335556,-86.8138056,842,-5,"U","America/New_York"',
 '6890,"Dowagiac Municipal Airport","Dowagiac","United States","C91",\\N,41.9929342,-86.1280125,748,-5,"U","America/New_York"']

In [19]:
def splitComma(line):
    splits = COMMA_DELIMITER.split(line)
    return splits[1], splits[2]

In [20]:
airportsNameAndCityNames = airportInUSA.map(splitComma)

In [21]:
airportsNameAndCityNames.take(2)

[('"Putnam County Airport"', '"Greencastle"'),
 ('"Dowagiac Municipal Airport"', '"Dowagiac"')]

In [22]:
airportsNameAndCityNames.saveAsTextFile('out/airports_in_usa.text')

#### Find all airports whose latitude are bigger than 40

In [29]:
airport_lat_more_40 = airport.filter(lambda line:float(COMMA_DELIMITER.split(line)[6]) > 40)

In [30]:
airport_lat_more_40.take(2)

['7,"Narsarsuaq","Narssarssuaq","Greenland","UAK","BGBW",61.160517,-45.425978,112,-3,"E","America/Godthab"',
 '8,"Nuuk","Godthaab","Greenland","GOH","BGGH",64.190922,-51.678064,283,-3,"E","America/Godthab"']

In [31]:
airportsNameAndCityNames = airport_lat_more_40.map(splitComma)

In [32]:
airportsNameAndCityNames.saveAsTextFile("out/airport_by_lat.txt")